In [1]:
import numpy as np
import pandas as pd
import shapely
import geopandas as geopandas
import fiona
import h3
import warnings
from shapely import geometry
from shapely.geometry import Polygon
warnings.filterwarnings("ignore")
import pyarrow.parquet as parquet
from geopy.geocoders import Nominatim

def style_fn(x):
    return {"color":"blue", "weight":3}

In [2]:
# Drop Z dimension of polygons that occurs often in KML
def _to_2d(x, y, z):
    return tuple(filter(None, [x, y]))

def remove_z(data:geopandas.GeoDataFrame):
    df = data.copy()
    list_geo = []

    # delete altitude calling _to_2d function
    for _, poli in df.iterrows():
        new_shape = shapely.ops.transform(_to_2d, poli.geometry)
        list_geo.append(new_shape)
    
    df.geometry = list_geo
    return df

In [3]:
maestro_cac=pd.read_csv('centros_alta_concurrencia.csv')

In [4]:
maestro_cac=maestro_cac[maestro_cac['latitude']!='None'].reset_index(drop=True)

In [5]:
maestro_cac['latitude']=maestro_cac['latitude'].astype(float)
maestro_cac['longitude']=maestro_cac['longitude'].astype(float)

In [6]:
from shapely.geometry import Point

crs = {'init': 'epsg:4326'}
maestro_cac['geometry']=maestro_cac.apply(lambda row: Point(row['longitude'],row['latitude']), axis=1)
maestro_cac = geopandas.GeoDataFrame(maestro_cac, crs = crs, geometry = maestro_cac.geometry)

In [7]:
maestro_cac = maestro_cac.to_crs("EPSG:32634")
maestro_cac.geometry = maestro_cac.geometry.buffer(1000, 1)

In [8]:
maestro_cac = maestro_cac.to_crs("EPSG:4326")

In [9]:
maestro_cac

,DEPARTAMENTO,PROVINCIA,DISTRITO,CLUSTER_CAT,CAC,NOMBRE_CAC,longitude,latitude,direccion,geometry
0,Amazonas,Chachapoyas,Chachapoyas,Principal,Plazas - Celdas Centro,Plaza - Amazonas,-77.899096,-6.308203,"Plaza de Armas Levanto, Levanto, Chachapoyas, ...","POLYGON ((-77.90036 -6.30925, -77.90016 -6.306..."
1,Ancash,Carhuaz,Marcara,Secundario,Aeropuertos,Aeropuerto Comandante FAP Germán Arias Graziani,-77.598292,-9.348183,"Comandante FAP Germán Arias Graziani Airport, ...","POLYGON ((-77.59957 -9.34974, -77.59985 -9.346..."
2,Ancash,Huaraz,Huaraz,Principal,Estadios,Estadio Rosas Pampa,-77.533541,-9.526551,"Estadio Rosas Pampa, Longitudinal de la Sierra...","POLYGON ((-77.53482 -9.52814, -77.53513 -9.525..."
3,Ancash,Santa,Chimbote,Principal,Estadios,Estadio Centenario Manuel Rivera Sanchez,-78.560437,-9.099858,Estadio Municipal Centenario Manuel Rivera Sán...,"POLYGON ((-78.56186 -9.10136, -78.56194 -9.098..."
4,Ancash,Santa,Chimbote,Principal,Estadios,Manuel Gomez Arellano,-78.589848,-9.064633,"Estadio Manuel Gómez Arellano, Avenida Industr...","POLYGON ((-78.59128 -9.06613, -78.59135 -9.063..."
...,...,...,...,...,...,...,...,...,...,...
184,Lima,Lima,Santa Rosa,Ancon,Playas-FdS,Playas-Santa Rosa _ Lima,-76.789172,-12.379692,"Santa Rosa, Punta Negra, Lima, Lima Metropolit...","POLYGON ((-76.79037 -12.38170, -76.79122 -12.3..."
185,Lima,Lima,Jesus Maria,Jesus Maria,Centro Comercial,Real Plaza Salaverry,-77.052211,-12.089856,"Real Plaza Salaverry, 2370, Avenida General Fe...","POLYGON ((-77.05344 -12.09182, -77.05421 -12.0..."
186,Lima,Lima,Independencia,Independencia,Centro Comercial,Plaza Norte,-77.058786,-12.006297,"Plaza Norte, 1400, Avenida Tomás Valle, Indepe...","POLYGON ((-77.06002 -12.00825, -77.06077 -12.0..."
187,Lima,Lima,San Miguel,San Miguel,Centro Comercial,Plaza San Miguel,-77.082713,-12.076927,"Plaza San Miguel, 2000, Avenida La Marina, Vir...","POLYGON ((-77.08395 -12.07889, -77.08471 -12.0..."


In [10]:
maestro_cac[~maestro_cac.is_empty]

,DEPARTAMENTO,PROVINCIA,DISTRITO,CLUSTER_CAT,CAC,NOMBRE_CAC,longitude,latitude,direccion,geometry
0,Amazonas,Chachapoyas,Chachapoyas,Principal,Plazas - Celdas Centro,Plaza - Amazonas,-77.899096,-6.308203,"Plaza de Armas Levanto, Levanto, Chachapoyas, ...","POLYGON ((-77.90036 -6.30925, -77.90016 -6.306..."
1,Ancash,Carhuaz,Marcara,Secundario,Aeropuertos,Aeropuerto Comandante FAP Germán Arias Graziani,-77.598292,-9.348183,"Comandante FAP Germán Arias Graziani Airport, ...","POLYGON ((-77.59957 -9.34974, -77.59985 -9.346..."
2,Ancash,Huaraz,Huaraz,Principal,Estadios,Estadio Rosas Pampa,-77.533541,-9.526551,"Estadio Rosas Pampa, Longitudinal de la Sierra...","POLYGON ((-77.53482 -9.52814, -77.53513 -9.525..."
3,Ancash,Santa,Chimbote,Principal,Estadios,Estadio Centenario Manuel Rivera Sanchez,-78.560437,-9.099858,Estadio Municipal Centenario Manuel Rivera Sán...,"POLYGON ((-78.56186 -9.10136, -78.56194 -9.098..."
4,Ancash,Santa,Chimbote,Principal,Estadios,Manuel Gomez Arellano,-78.589848,-9.064633,"Estadio Manuel Gómez Arellano, Avenida Industr...","POLYGON ((-78.59128 -9.06613, -78.59135 -9.063..."
...,...,...,...,...,...,...,...,...,...,...
184,Lima,Lima,Santa Rosa,Ancon,Playas-FdS,Playas-Santa Rosa _ Lima,-76.789172,-12.379692,"Santa Rosa, Punta Negra, Lima, Lima Metropolit...","POLYGON ((-76.79037 -12.38170, -76.79122 -12.3..."
185,Lima,Lima,Jesus Maria,Jesus Maria,Centro Comercial,Real Plaza Salaverry,-77.052211,-12.089856,"Real Plaza Salaverry, 2370, Avenida General Fe...","POLYGON ((-77.05344 -12.09182, -77.05421 -12.0..."
186,Lima,Lima,Independencia,Independencia,Centro Comercial,Plaza Norte,-77.058786,-12.006297,"Plaza Norte, 1400, Avenida Tomás Valle, Indepe...","POLYGON ((-77.06002 -12.00825, -77.06077 -12.0..."
187,Lima,Lima,San Miguel,San Miguel,Centro Comercial,Plaza San Miguel,-77.082713,-12.076927,"Plaza San Miguel, 2000, Avenida La Marina, Vir...","POLYGON ((-77.08395 -12.07889, -77.08471 -12.0..."


In [11]:
import folium
map_test = folium.Map(tiles="cartodbpositron")

folium.GeoJson(
    maestro_cac[~maestro_cac.is_empty],
    style_function=style_fn,
    popup=folium.GeoJsonPopup(['Departamento'.upper(),'Provincia'.upper(),'Distrito'.upper(),'NOMBRE_CAC']),
    tooltip=folium.GeoJsonTooltip(['Departamento'.upper(),'Provincia'.upper(),'Distrito'.upper(),'NOMBRE_CAC']),
).add_to(map_test)

map_test